In [ ]:
def get_state(state):
  state = np.array(state)
  state = np.moveaxis(state,0,2)/255 #normalisation
  state = np.expand_dims(state,0)
  state = torch.from_numpy(state)
  state = state.clone().detach().to(dtype=torch.float32, device=device).unsqueeze(0)
  state = torch.permute(state, (0,1,4,3,2))
  state = torch.squeeze(state)
  return state

In [ ]:
def resize_frame(frame):
    frame = frame[30:-12,5:-4]
    frame = np.average(frame,axis = 2)
    frame = cv2.resize(frame,(84,84),interpolation = cv2.INTER_NEAREST)
    frame = np.array(frame,dtype = np.uint8)
    return frame

In [ ]:
def update_target_network(policy_net, target_net, TAU):
    target_net_state_dict = target_net.state_dict()
    policy_net_state_dict = policy_net.state_dict()
    for key in policy_net_state_dict:
        target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
    target_net.load_state_dict(target_net_state_dict)

In [ ]:
def agent_explore(state):
    global steps_done
    sample = random.random()
    state = state.clone().detach().to(dtype=torch.float32, device=device).unsqueeze(0)
    eps_threshold = 0.1 + (1 - 0.1) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

In [ ]:
def agent_exploit(state):
    state = state.unsqueeze(0)
    print(policy_net(state))
    action = policy_net(state).max(1)[1].view(1, 1)
    return action

In [ ]:
def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(scores, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())